# Process DEA PDF
<br>
James Chapman<br>
CIS 830 Advanced Topics in AI – Term Project<br>
Kansas State University<br>

This notebook simply extracts the synonym/slang terms from the following DEA PDF.


[Slang Terms and Code Words:  A Reference for Law Enforcement Personnel](https://www.dea.gov/sites/default/files/2018-07/DIR-022-18.pdf)

Creates dataframe and saves as CSV - "drug_slang.csv"<br><br><br>
for each reference drug<br>
"index_term" -> "alt_term" (List of synonyms/slang terms)

In [1]:
import csv
import re
import ftfy
import pandas as pd
from PyPDF2 import PdfReader
from tqdm import tqdm
tqdm.pandas()

In [2]:
# https://www.dea.gov/sites/default/files/2018-07/DIR-022-18.pdf
pdf_path = '../data/DIR-022-18 DEA Slang Terms and Code Words  A Reference for Law Enforcement Personnel.pdf'

# PDF -> text

In [3]:
def extract_text_from_pdf(pdf_path):
    """Extract all text from a PDF file."""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

text = extract_text_from_pdf(pdf_path)
print(text)

UNCLASSIFIED
UNCLASSIFIED1DEA-HOU-DIR-022-18Slang Terms and Code 
Words:  A Reference for Law 
Enforcement PersonnelDEA  
Intelligence  
BriefJuly 2018
DEA  
Intelligence  
Report

UNCLASSIFIED
UNCLASSIFIED2
DEA Intelligence ReportExecutive Summary
This Drug Enforcement Administration (DEA) Intelligence Report contains new and updated information on 
slang terms and code words from a variety of law enforcement and open sources, and serves as an updated 
version to the product entitled “Drug Slang Code Words” published by the DEA in May 2017.  It is designed 
as a ready reference for law enforcement personnel who are confronted with hundreds of slang terms and 
code words used to identify a wide variety of controlled substances, designer drugs, synthetic compounds, 
measurements, locations, weapons, and other miscellaneous terms relevant to the drug trade.  Although 
every effort was made to ensure the accuracy and completeness of the information presented, due to the 
dynamics of the e

# Section 1

In [4]:

def extract_drug_slang_section(text):
    """
    Extract the part of the text that contains the "Drug Slang Terms and Code Words" section.
    It assumes that the section starts with a header (e.g. "Drug Slang Terms and Code Words")
    and continues until a marker like "See Appendix A" appears.
    """
    pattern = r"Drug Slang Terms and Code Words.*?(?=Additional Slang Terms and Code Words)"
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(0)
    else:
        print("Could not find the Drug Slang section.")
        return ""
    
# Section 1: Drug Slang Terms and Code Words
def parse_section_1(section_text):
    """
    Parse the drug slang section to build records.
    Each record is expected to have:
         - A header line (the official drug name, which often includes parentheses)
         - Followed by one or more lines that list slang terms separated by semicolons.
    This version accounts for trailing terms where a slang term may span two lines.
    If a slang line does not end with a semicolon and the next non-blank line contains a semicolon,
    it will be merged into the current slang line. Otherwise, the line is considered complete.
    
    Returns a list of dicts with keys "index_term" and "alt_term".
    """
    lines = section_text.splitlines()
    records = []
    current_drug = None
    current_slangs = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        if not line:
            i += 1
            continue  # skip blank lines

        # If the line does not contain a semicolon, assume it is a new header
        if ";" not in line:
            # Before starting a new record, save the previous one (if any)
            if current_drug is not None and current_slangs:
                record = {
                    "index_term": current_drug,
                    "alt_term": "; ".join(current_slangs)
                }
                records.append(record)
            # The new header becomes the current drug
            current_drug = line
            current_slangs = []
            i += 1
        else:
            # Process a line with slang terms.
            # We want to join the line with the following one if it doesn't end with ';'
            current_line = line
            # Lookahead to merge if the line seems to be broken
            while (not current_line.rstrip().endswith(";") and (i + 1) < len(lines)):
                # Peek at the next non-blank line
                j = i + 1
                next_line = ""
                while j < len(lines) and not next_line:
                    next_line = lines[j].strip()
                    if not next_line:
                        j += 1
                # If next_line exists and looks like a slang list (contains a semicolon)
                # then join it to the current_line.
                if next_line and ";" in next_line:
                    current_line += " " + next_line
                    i = j  # skip over the merged line(s)
                else:
                    break

            # Split the merged line by semicolons and add non-empty terms to the current slang list.
            parts = [part.strip() for part in current_line.split(";") if part.strip()]
            current_slangs.extend(parts)
            i += 1

    # Add the last record if available
    if current_drug is not None and current_slangs:
        record = {
            "index_term": current_drug,
            "alt_term": "; ".join(current_slangs)
        }
        records.append(record)
    return records

def write_csv(filename, records, fieldnames):
    """Write records (list of dictionaries) to a CSV file using the specified fieldnames."""
    with open(filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for rec in records:
            writer.writerow(rec)



In [5]:
original_drug_slang_text = extract_drug_slang_section(text)
print(original_drug_slang_text)

# with open('drug_slang_text.txt', 'w', encoding='utf-8') as file:
#     file.write(drug_slang_text)
"""
   ##############################################
   I opened a text editor and deleted some lines by hand !
   ###############################################
"""
with open('../data/drug_slang_text.txt', 'r', encoding='utf-8-sig') as file:
    drug_slang_text = file.read()


Drug Slang Terms and Code Wordsa
Acetaminophen and Oxycodone Combination (Percocet®)
512s; Bananas; Blue; Blue Dynamite;  Blueberries; Buttons; Ercs; Greenies; Hillbilly Heroin; Kickers; M-30s; 
Paulas ; Percs; Rims; Tires; Wheels
Alprazolam (Xanax® )
Bars; Benzos; Bicycle Handle Bars; Bicycle Parts ; Bricks ; Footballs; Handlebars ; Hulk; L7; Ladders; Palitroque ; 
Planks; School Bus; Sticks; Upjohns ; White Boys ; White Girls ; Xanies; Yellow Boys ; Zanbars; Zannies; Z-Bars
Amphetamine
Acelerador ; Amy; Amps; Bam; B-Bombs; Beans; Bennies; Benz; Black and Whites; Black Beauties; Black 
Birds; Black Bombers; Black Mollies; Blacks; Blue Boys; Bombita; Brain Ticklers; Brownies; Bumblebees; 
Cartwheels; Chalk; Chicken Powder; Chochos; Chocolates ; Christina; Chunk; Co-Pilot; Coast-to-Coasts; 
Crisscross; Cross Roads; Cross Tops; Crosses ; Debs; Dexies; Diablos; Diamonds; Diet Pills; Dolls; Dominoes; 
Double Cross; Drivers; Dulces ; Fives; Flour ; Footballs; French Blues; Geeked Up ; Goofb

In [6]:
print(drug_slang_text)


Acetaminophen and Oxycodone Combination (Percocet®)
512s; Bananas; Blue; Blue Dynamite;  Blueberries; Buttons; Ercs; Greenies; Hillbilly Heroin; Kickers; M-30s; 
Paulas ; Percs; Rims; Tires; Wheels
Alprazolam (Xanax®)
Bars; Benzos; Bicycle Handle Bars; Bicycle Parts ; Bricks ; Footballs; Handlebars ; Hulk; L7; Ladders; Palitroque ; 
Planks; School Bus; Sticks; Upjohns ; White Boys ; White Girls ; Xanies; Yellow Boys ; Zanbars; Zannies; Z-Bars
Amphetamine
Acelerador ; Amy; Amps; Bam; B-Bombs; Beans; Bennies; Benz; Black and Whites; Black Beauties; Black 
Birds; Black Bombers; Black Mollies; Blacks; Blue Boys; Bombita; Brain Ticklers; Brownies; Bumblebees; 
Cartwheels; Chalk; Chicken Powder; Chochos; Chocolates ; Christina; Chunk; Co-Pilot; Coast-to-Coasts; 
Crisscross; Cross Roads; Cross Tops; Crosses ; Debs; Dexies; Diablos; Diamonds; Diet Pills; Dolls; Dominoes; 
Double Cross; Drivers; Dulces ; Fives; Flour ; Footballs; French Blues; Geeked Up ; Goofballs; Greenies; Head 
Drugs; Hear

In [7]:
csv1_records = parse_section_1(drug_slang_text)
write_csv('../data/DEA_slang.csv', csv1_records, ["index_term", "alt_term"])
print(csv1_records)

[{'index_term': 'Acetaminophen and Oxycodone Combination (Percocet®)', 'alt_term': '512s; Bananas; Blue; Blue Dynamite; Blueberries; Buttons; Ercs; Greenies; Hillbilly Heroin; Kickers; M-30s; Paulas; Percs; Rims; Tires; Wheels'}, {'index_term': 'Alprazolam (Xanax®)', 'alt_term': 'Bars; Benzos; Bicycle Handle Bars; Bicycle Parts; Bricks; Footballs; Handlebars; Hulk; L7; Ladders; Palitroque; Planks; School Bus; Sticks; Upjohns; White Boys; White Girls; Xanies; Yellow Boys; Zanbars; Zannies; Z-Bars'}, {'index_term': 'Amphetamine', 'alt_term': 'Acelerador; Amy; Amps; Bam; B-Bombs; Beans; Bennies; Benz; Black and Whites; Black Beauties; Black Birds; Black Bombers; Black Mollies; Blacks; Blue Boys; Bombita; Brain Ticklers; Brownies; Bumblebees; Cartwheels; Chalk; Chicken Powder; Chochos; Chocolates; Christina; Chunk; Co-Pilot; Coast-to-Coasts; Crisscross; Cross Roads; Cross Tops; Crosses; Debs; Dexies; Diablos; Diamonds; Diet Pills; Dolls; Dominoes; Double Cross; Drivers; Dulces; Fives; Flou

# Section 2<br>

## *** Did not work - This section is a sorted table representation of section 1


In [8]:

def extract_appendix_section(text):
    """
    Extract the appendix section. This assumes that the text starting from the header
    "Appendix A.  Alphabetized List of Slang Terms and Code Words For Common Drugs" up to the end.
    """
    pattern = r"Appendix A\.\s*Alphabetized List of Slang Terms and Code Words For Common Drugs(.*)"
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        print("Could not find the Appendix section.")
        return ""

# Section 2: Appendix A.  Alphabetized List of Slang Terms and Code Words For Common Drugs
def parse_section_2(appendix_text):
    """
    Parse the Appendix section where each line is in the form:
         slang_or_code_word  Associated Drug(s)
    (e.g. "7 Cocaine" or "512s Acetaminophen and Oxycodone Combination (Percocet®); Oxycodone (Oxycontin®, Roxicodone®, Oxaydo®)")
    Splits the line at the first whitespace.
    Returns a list of dicts with keys "alt_term" and "index_term".
    """
    records = []
    # Split the appendix text into lines and ignore any header lines that do not conform.
    lines = appendix_text.splitlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Use regex to split into two parts: the slang (first token) and the rest as associated drug(s)
        m = re.match(r"(\S+)\s+(.*)", line)
        if m:
            slang = m.group(1).strip()
            drug = m.group(2).strip()
            records.append({"alt_term": slang, "index_term": drug})
    return records




In [9]:
"""
    Appendix data didn't work as expected because of the spacing
"""
appendix_text = extract_appendix_section(text)
print(appendix_text)



Slang/Code Word Associated Drug(s)
7 Cocaine
30s Oxycodone (Oxycontin®, Roxicodone®, Oxaydo®)
40s Oxycodone (Oxycontin®, Roxicodone®, Oxaydo®)
51s Crack Cocaine
62 Cocaine
77 Cocaine
151s Crack Cocaine
357s Hydrocodone (Norco®, Vicodin®, Lorcet®)
420 Marijuana; Synthetic Cannabinoids
501s Crack Cocaine
512sAcetaminophen and Oxycodone Combination (Percocet®); Oxycodone 
(Oxycontin®, Roxicodone®, Oxaydo®)
542 Flunitrazepam (Rohypnol®)
710 Marijuana Concentrates/Hash Oil
777 Cocaine
921 Cocaine
A-1 Cocaine
UNCLASSIFIED
UNCLASSIFIED14
DEA Intelligence ReportAbajo Heroin
Abby Amphetamine and Dextroamphetamine Combination (Adderall®)
A-Bomb Heroin mixed with Marijuana
Abyss Synthetic Cannabinoids
Abyssian Tea Khat
Acapulco Gold Marijuana
Acapulco Red Marijuana
Accordion Methamphetamine
Ace Marijuana; PCP (Phencyclidine)
Ace of Spades Synthetic Cannabinoids
Aceite LSD (Lysergic Acid Diethylamide)
Acelerador Amphetamine
Acelide LSD (Lysergic Acid Diethylamide)
Achivia Heroin
Acid LSD (Lysergi

In [10]:
# csv2_records = parse_section_2(appendix_text)
# write_csv("appendix_slang.csv", csv2_records, ["alt_term", "index_term"])
# print(csv2_records)